In [57]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

INPUT_SIZE = 784
OUTPUT_SIZE = 10
HIDDEN_SIZE = 512
BATCH_SIZE = 128
LEARNING_RATE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 10000
MOVING_AVERAGE_DECAY = 0.99

In [60]:
# def inference(input_tensor, avg_class, w1, b1, w2, b2):
#     if avg_class == None:
#         layer1 = tf.nn.relu(tf.matmul(input_tensor, w1) + b1)
#         # softmax is in loss function
#         layer2 = tf.matmul(layer1, w2) + b2
#         return layer2
#     else:
#         layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(w1)) + avg_class.average(b1))
#         layer2 = tf.matmul(layer1, avg_class.average(w2)) + avg_class.average(b2)
#         return layer2

def inference(input_tensor):
    with tf.variable_scope("layer1", reuse=tf.AUTO_REUSE):
        weights = tf.get_variable("weights", [INPUT_SIZE, HIDDEN_SIZE], 
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", [HIDDEN_SIZE], 
                                  initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
    
    with tf.variable_scope("layer2", reuse=tf.AUTO_REUSE):
        weights = tf.get_variable("weights", [HIDDEN_SIZE, OUTPUT_SIZE], 
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", [OUTPUT_SIZE], 
                                  initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    return layer2

def train(mnist):
    feature = tf.placeholder(tf.float32, [None, INPUT_SIZE], name="feature")
    label = tf.placeholder(tf.float32, [None, OUTPUT_SIZE], name="label")
#     w1 = tf.Variable(tf.truncated_normal([INPUT_SIZE, HIDDEN_SIZE], stddev=0.1))
#     b1 = tf.Variable(tf.constant(0., shape=[HIDDEN_SIZE]))
#     w2 = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, OUTPUT_SIZE], stddev=0.1))
#     b2 = tf.Variable(tf.constant(0., shape=[OUTPUT_SIZE]))
    #     y = inference(feature, None, w1, b1, w2, b2)
    y = inference(feature)
    
    # use moving averages
    global_step = tf.Variable(0, trainable=False)
#     variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
#     variable_averages_op = variable_averages.apply(tf.trainable_variables())
#     y_average = inference(feature, variable_averages, w1, b1, w2, b2) 
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(label,1), logits=y)
    reg = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    loss = tf.reduce_mean(cross_entropy) + reg(w1) + reg(w2)
    learning_rate = tf.train.exponential_decay(LEARNING_RATE, global_step, mnist.train.num_examples/BATCH_SIZE, 
                                           LEARNING_RATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    train_op = tf.group([train_step, variable_averages_op])
    
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for i in range(TRAINING_STEPS):
            data_x, data_y = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={feature: data_x, label: data_y})
        
            if i%1000==0:
                valid_acc = sess.run(accuracy, feed_dict={feature: mnist.validation.images,
                                                      label: mnist.validation.labels})
                test_acc = sess.run(accuracy, feed_dict={feature: mnist.test.images,
                                                      label: mnist.test.labels})
                print("Valid acc at step {}: {}".format(i, valid_acc))
                print("Test acc at step {}: {}".format(i, test_acc))

In [61]:
mnist = input_data.read_data_sets("mnist_data/",one_hot=True)
train(mnist)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
Valid acc at step 0: 0.2937999963760376
Test acc at step 0: 0.28029999136924744
Valid acc at step 1000: 0.9757999777793884
Test acc at step 1000: 0.9740999937057495
Valid acc at step 2000: 0.9787999987602234
Test acc at step 2000: 0.9778000116348267
Valid acc at step 3000: 0.978600025177002
Test acc at step 3000: 0.9782999753952026
Valid acc at step 4000: 0.9836000204086304
Test acc at step 4000: 0.98089998960495
Valid acc at step 5000: 0.9828000068664551
Test acc at step 5000: 0.9814000129699707
Valid acc at step 6000: 0.9833999872207642
Test acc at step 6000: 0.9825999736785889
Valid acc at step 7000: 0.984000027179718
Test acc at step 7000: 0.9818999767303467
Valid acc at step 8000: 0.9833999872207642
Test acc at step 8000: 0.9822999835014343
Valid acc at step 9000: 0.9836000204086304
Test acc at step 9000: 0.98